In [ ]:
!pip install langchain
!pip install langchain-openai
!pip install langchain_core
!pip install langchain_community

!pip install langchain-google-genai
!pip install google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 30.3 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.2 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.18
    Uninstalling google-ai-generativelanguage-0.6.18:
      Successfully uninstalled google-ai-generativelanguage-0.6.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.7 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


## LCEL - Pipe operator and runnables

There are two ways to use Pipe
- **`|`** - the pipe operator
- **`.pipe()`** - function

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["GOOGLE_API_KEY"] = ""

In [ ]:
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

dictionary_template = PromptTemplate(
    input_variables=["word"],
    template = "Give me one line definition of {word}. Response should not contain the  - {word} itself, only the meaning."
)

llm = ChatOpenAI()

# the pipe operator
chain1 = dictionary_template | llm | StrOutputParser()

In [ ]:
chain1.invoke({"word":"Onomatopoeia"})

'Onomatopoeia is the formation of a word from a sound associated with what is named.'

In [ ]:
guesser_template = PromptTemplate(
    input_variables=["definition"],
    template = "Give me an English word which has the following meaning - {definition}."
)

llm = ChatOpenAI()

# the pipe operator
chain2 = guesser_template.pipe(llm).pipe(StrOutputParser())

In [ ]:
chain2.invoke("The use of words that imitate sounds.")

'Onomatopoeia'

In [ ]:
seq_chain = {"definition": chain1} | chain2
seq_chain.invoke({"word":"Onomatopoeia"})

'Buzz.'

In [ ]:
seq_chain = {"definition": chain1} | chain2
seq_chain.invoke({"word":"Sesquipedalian"})

'Verbose'

## Runnables

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel

In [ ]:
chain = RunnablePassthrough()

In [ ]:
chain.invoke("input") # input will be passed through the chian

'input'

In [ ]:
def output_length(input: str):
  output = len(input)
  return output

# RunnableLambda runs a function on the input
chain = RunnableLambda(output_length)
chain.invoke("input")

5

In [ ]:
chain = RunnableParallel(text = RunnablePassthrough(), length = RunnableLambda(output_length))

chain.invoke("startech academy")

{'text': 'startech academy', 'length': 16}

## Example

In [ ]:
from langchain import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

explainer_template = PromptTemplate(
    input_variables= ["topic"],
    template= "Explain this topic in a single paragraph as if you were explaining it to a 10 year old - {topic}."
)

openAIllm = ChatOpenAI()

OpenAI_chain= explainer_template | openAIllm | StrOutputParser()
OpenAI_chain.invoke({"topic":"Gravity"})

"Gravity is like a big invisible force that pulls everything towards the Earth. It's what keeps us from floating away into space. Objects with more weight have stronger gravity, so they pull things towards them more than lighter objects. That's why when you drop something, it falls down instead of floating up!"

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

explainer_template = PromptTemplate(
    input_variables= ["topic"],
    template= "Explain this topic in a single paragraph as if you were explaining it to a 10 year old - {topic}."
)

google_chain = ChatGoogleGenerativeAI(model="models/gemini-1.5-pro")

google_chain= explainer_template | google_chain | StrOutputParser()
google_chain.invoke({"topic":"Gravity"})

"Imagine you're holding a ball and then let go.  It falls to the ground, right? That's gravity! It's an invisible force that pulls everything towards each other.  The bigger something is, the stronger its gravity.  That's why the Earth pulls you towards it and keeps you on the ground, and why the Earth goes around the even bigger Sun.  Even you have gravity, but you're so small that your pull is too weak to notice!"

In [ ]:
analyzer_template = PromptTemplate(
    input_variables = ["topic", "explanation1", "explanation2"],
    template = """
    Which of the two explanations given below are better for explaining  {topic} to a 10 year old student.
    ###
    Explanation1 - {explanation1}
    ###
    ***
    Explanation2 - {explanation2}
    ***
    """
)

combined_chain = (
    RunnableParallel({"explanation1": OpenAI_chain, "explanation2": google_chain, "topic": RunnablePassthrough()})
    | analyzer_template
    | openAIllm
    | StrOutputParser()
)

combined_chain.invoke({"topic": "Gravity"})

'Explanation 2 is better for explaining gravity to a 10 year old student. It uses simpler language and examples that are easier for a child to understand, such as holding a ball and letting it fall to the ground. It also includes a relatable comparison by saying gravity is like an "invisible hug from everything in the universe," which can help the student grasp the concept more easily.'

In [ ]:
analyzer_chain = analyzer_template | openAIllm | StrOutputParser()

updated_chain = (
    RunnableParallel({"explanation1": OpenAI_chain, "explanation2": google_chain, "topic": RunnablePassthrough()})
    | RunnableParallel({"input": RunnablePassthrough(), "analysis": analyzer_chain})
)

updated_chain.invoke({"topic": "Gravity"})

{'input': {'explanation1': "Gravity is a force that pulls things toward each other, like when you drop a ball and it falls to the ground. It's what keeps us on the earth and makes things stay in place instead of floating away. Gravity is what makes apples fall from trees and why we have to push off the ground to jump up in the air. It's a natural force that keeps everything from floating away into space.",
  'explanation2': "Imagine you're holding a ball and then let go.  It falls to the ground, right? That's gravity!  It's an invisible force that pulls everything towards each other.  The Earth is really big and heavy, so it has a strong pull, which keeps us stuck to the ground and makes the ball fall.  The Sun is even bigger, so its gravity keeps all the planets spinning around it.  Even you have gravity, but you're so small that your pull isn't strong enough to notice.  Basically, the bigger something is, the stronger its gravity.",
  'topic': {'topic': 'Gravity'}},
 'analysis': 'Exp